In [1]:
import os
import numpy as np
import cv2
import scipy.io
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Dropout, Conv2DTranspose, Add, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU

C:\Users\noorh\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\noorh\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.19.1) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [2]:
drop = 0.25

def ResBlock(input_tensor, filters):
    conv_1 = Conv2D(filters=filters, kernel_size=3, padding='same', kernel_initializer='he_normal')    
    conv_1a = conv_1(input_tensor) # Shared weights conv layer
    batch_1 = BatchNormalization()(conv_1a)
    relu_1 = Activation("relu")(batch_1)
    drop_1 = Dropout(drop)(relu_1)
    conv_1b = conv_1(drop_1) # Shared weights conv layer
    batch_1 = BatchNormalization()(conv_1b)
    return batch_1

def LadderNet(input_size=(224, 224, 3), num_classes=2, filters=30):
    inputs = Input(input_size)
    X = Conv2D(filters=filters, kernel_size=3, activation="relu", padding='same', kernel_initializer='he_normal')(inputs)

    X1 = ResBlock(input_tensor=X, filters=filters)
    X = Conv2D(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X1) 
    X = Activation("relu")(X)
    
    X2 = ResBlock(input_tensor=X, filters=filters*2)
    X = Conv2D(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X2)
    X = Activation("relu")(X)
    
    X3 = ResBlock(input_tensor=X, filters=filters*4)
    X = Conv2D(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X3)
    X = Activation("relu")(X)
    
    X4 = ResBlock(input_tensor=X, filters=filters*8)
    X = Conv2D(filters=filters*16, kernel_size=3, strides=2, kernel_initializer='he_normal')(X4)
    X = Activation("relu")(X)
    
    X = ResBlock(input_tensor=X, filters=filters*16)
    
    X = Conv2DTranspose(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X4])
    X5 = ResBlock(input_tensor=X, filters=filters*8)
    
    X = Conv2DTranspose(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X5)
    X = Add()([X, X3])
    X6 = ResBlock(input_tensor=X, filters=filters*4)
    
    X = Conv2DTranspose(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X6)
    X = Add()([X, X2])
    X7 = ResBlock(input_tensor=X, filters=filters*2)
        
    X = Conv2DTranspose(filters=filters, kernel_size=3, strides=2, output_padding=1, kernel_initializer='he_normal')(X7)
    X = Add()([X, X1])
    X = ResBlock(input_tensor=X, filters=filters)
    
    X8 = ResBlock(input_tensor=X, filters=filters)
    X = ResBlock(input_tensor=X, filters=filters)
    X = Add()([X, X8])
    
    X9 = ResBlock(input_tensor=X, filters=filters)
    X = Conv2D(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X) 
    X = Activation("relu")(X)
    X = Add()([X7, X])    
    
    X10 = ResBlock(input_tensor=X, filters=filters*2)
    X = Conv2D(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)    
    X = Add()([X6, X])
    
    X11 = ResBlock(input_tensor=X, filters=filters*4)
    X = Conv2D(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)
    X = Add()([X5, X])

    X12 = ResBlock(input_tensor=X, filters=filters*8)
    X = Conv2D(filters=filters*16, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)
    
    X = ResBlock(input_tensor=X, filters=filters*16)
    
    X = Conv2DTranspose(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X12])   
    X = ResBlock(input_tensor=X, filters=filters*8)
    
    X = Conv2DTranspose(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X11])
    X = ResBlock(input_tensor=X, filters=filters*4)
    
    X = Conv2DTranspose(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X10])
    X = ResBlock(input_tensor=X, filters=filters*2)
    
    X = Conv2DTranspose(filters=filters, kernel_size=3, strides=2, kernel_initializer='he_normal', output_padding=1)(X)
    X = Add()([X, X9])
    X = ResBlock(input_tensor=X, filters=filters)
    
    X = Conv2D(filters=num_classes, kernel_size=1, kernel_initializer='he_normal')(X)
    X = Activation("softmax")(X)
    
    model = Model(inputs, X)
    
    return model

In [3]:
def load_data(mat_dir, frame_dir):
    mat_files = [f for f in os.listdir(mat_dir) if f.endswith('.mat')]
    frames = []
    masks = []
    
    for mat_file in mat_files:
        video_name = os.path.splitext(mat_file)[0].replace("Mask_", "")
        mat_path = os.path.join(mat_dir, mat_file)
        frames_path = os.path.join(frame_dir, video_name)
        
        if os.path.exists(frames_path):
            mat_data = scipy.io.loadmat(mat_path)['predicted']
            num_masks = mat_data.shape[0]
            for i in range(num_masks):
                mask = cv2.resize(mat_data[i], (224, 224))
                mask = np.expand_dims(mask, axis=-1)  # Expand dimensions to make it (224, 224, 1)
                mask = mask / np.max(mask)
                mask = (mask > 0.5).astype(np.uint8)  # Ensure masks are binary
                
                frame_name = f'frame_{i+1}.png' 
                frame_path = os.path.join(frames_path, frame_name)
                if os.path.exists(frame_path):
                    frame = cv2.imread(frame_path)
                    frame = cv2.resize(frame, (224, 224))
                    frame = frame / 255.0
                    frames.append(frame)
                    masks.append(mask)
                else:
                    print(f"Frame {frame_path} not found.")
    
    return np.array(frames), np.array(masks)

def calculate_metrics(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    
    return sensitivity, specificity, accuracy, precision

In [6]:
mat_dir = r"C:\Users\noorh\OneDrive\Desktop\Grad Project\HMC Dataset\LV Ground-truth Segmentation Masks"
frame_dir = r"C:\Users\noorh\OneDrive\Desktop\Grad Project\HMC Dataset\Extracted Frames"
save_dir = r"C:\Users\noorh\OneDrive\Desktop\Grad Project\HMC Dataset"
os.makedirs(save_dir, exist_ok=True)

frames, masks = load_data(mat_dir, frame_dir)
    
X_train, X_test, y_train, y_test = train_test_split(frames, masks, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
model = LadderNet(input_size=(224, 224, 3), num_classes=3, filters=30)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_val, y_val))
model_path = os.path.join(save_dir, 'laddernet_model.keras')
model.save(model_path)




Epoch 1/50


188/188 [==============================] - 1213s 6s/step - loss: 0.3855 - accuracy: 0.9449 - val_loss: 0.1761 - val_accuracy: 0.9789
Epoch 2/50
188/188 [==============================] - 1185s 6s/step - loss: 0.0443 - accuracy: 0.9921 - val_loss: 0.0332 - val_accuracy: 0.9927
Epoch 3/50
188/188 [==============================] - 1190s 6s/step - loss: 0.0238 - accuracy: 0.9934 - val_loss: 0.0240 - val_accuracy: 0.9940
Epoch 4/50
188/188 [==============================] - 1150s 6s/step - loss: 0.0198 - accuracy: 0.9935 - val_loss: 0.0168 - val_accuracy: 0.9943
Epoch 5/50
188/188 [==============================] - 1135s 6s/step - loss: 0.0155 - accuracy: 0.9944 - val_loss: 0.0143 - val_accuracy: 0.9945
Epoch 6/50
188/188 [==============================] - 1133s 6s/step - loss: 0.0143 - accuracy: 0.9946 - val_loss: 0.0132 - val_accuracy: 0.9948
Epoch 7/50
188/188 [==============================] - 1140s 6s/step - loss: 0.0128 - accuracy: 0.9950 - val_loss: 0.0195 - val_accu

Epoch 50/50
188/188 [==============================] - 1231s 7s/step - loss: 0.0042 - accuracy: 0.9982 - val_loss: 0.0053 - val_accuracy: 0.9979


In [7]:
predictions = model.predict(X_test)
iou_metric = MeanIoU(num_classes=2)
iou_metric.update_state(y_test, np.argmax(predictions, axis=-1))
print("Mean IoU =", iou_metric.result().numpy())

15/15 [==============================] - 73s 5s/step
Mean IoU = 0.9731815


In [8]:
y_test_flat = y_test.flatten()
y_pred_flat = np.argmax(predictions, axis=-1).flatten()
    
sensitivity, specificity, accuracy, precision = calculate_metrics(y_test_flat, y_pred_flat)
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")

Sensitivity: 0.973091022303074
Specificity: 0.9989144176216029
Accuracy: 0.9978381628582284
Precision: 0.9749896784256159


In [9]:
import pandas as pd

history_df = pd.DataFrame(history.history)
history_df.to_csv(r"C:\Users\noorh\Echocardiograms\Final Submission/history.csv", index=False)